In [3]:
import dspy
import openai

openai.api_key = "sk-foobar"

gpt4 = dspy.OpenAI(model="gpt-4", max_tokens=4000, model_type="chat")

llama3_ollama = dspy.OllamaLocal(
    model="llama3:instruct",
    max_tokens=4000,
    timeout_s=480
)

dspy.settings.configure(lm=llama3_ollama)

print(llama3_ollama("say hello"))
print(gpt4("say hello"))

["Hello! It's nice to meet you. Is there something I can help you with, or would you like to chat?"]
['Hello! How can I assist you today?']


In [4]:
# Load blogs into Weaviate
import weaviate

weaviate_client = weaviate.connect_to_local()

weaviate_client.collections.delete("WeaviateBlogChunk")

In [5]:
import weaviate.classes.config as wvcc

collection = weaviate_client.collections.create(
   name="WeaviateBlogChunk",
   vectorizer_config=wvcc.Configure.Vectorizer.text2vec_ollama
   (
       api_endpoint="http://host.docker.internal:11434",
       model="snowflake-arctic-embed:335m"
   ),
   properties=[
           wvcc.Property(name="content", data_type=wvcc.DataType.TEXT),
           wvcc.Property(name="inspired_question", data_type=wvcc.DataType.TEXT, skip_vectorization=True)
     ]
)

In [6]:
import os
import re


def chunk_list(lst, chunk_size):
    """Break a list into chunks of the specified size."""
    return [lst[i:i + chunk_size] for i in range(0, len(lst), chunk_size)]


def split_into_sentences(text):
    """Split text into sentences using regular expressions."""
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text)
    return [sentence.strip() for sentence in sentences if sentence.strip()]


def read_and_chunk_index_files(main_folder_path):
    """Read index.md files from subfolders, split into sentences, and chunk every 5 sentences."""
    blog_chunks = []
    for folder_name in os.listdir(main_folder_path):
        subfolder_path = os.path.join(main_folder_path, folder_name)
        if os.path.isdir(subfolder_path):
            index_file_path = os.path.join(subfolder_path, 'index.mdx')
            if os.path.isfile(index_file_path):
                with open(index_file_path, 'r', encoding='utf-8') as file:
                    content = file.read()
                    sentences = split_into_sentences(content)
                    sentence_chunks = chunk_list(sentences, 5)
                    sentence_chunks = [' '.join(chunk) for chunk in sentence_chunks]
                    blog_chunks.extend(sentence_chunks)
    return blog_chunks


# Example usage
main_folder_path = '../datasets/weaviate-blogs'
blog_chunks = read_and_chunk_index_files(main_folder_path)


print(f"{len(blog_chunks)}\n")
print(blog_chunks[0])

1182

---
title: Combining LangChain and Weaviate
slug: combining-langchain-and-weaviate
authors: [erika]
date: 2023-02-21
tags: ['integrations']
image: ./img/hero.png
description: "LangChain is one of the most exciting new tools in AI. It helps overcome many limitations of LLMs, such as hallucination and limited input lengths."
---
![Combining LangChain and Weaviate](./img/hero.png)

Large Language Models (LLMs) have revolutionized the way we interact and communicate with computers. These machines can understand and generate human-like language on a massive scale. LLMs are a versatile tool that is seen in many applications like chatbots, content creation, and much more. Despite being a powerful tool, LLMs have the drawback of being too general.


In [7]:
from weaviate.util import get_valid_uuid
from uuid import uuid4
import time

blogs = weaviate_client.collections.get("WeaviateBlogChunk")

blog_chunk_uuids = []

start = time.time()
for idx, blog_chunk in enumerate(blog_chunks):
    id = get_valid_uuid(uuid4())
    blog_chunk_uuids.append(id)
    upload = blogs.data.insert(
        properties={
           "content": blog_chunk
        },
        uuid=id
    )

print(f"Uploaded and vectorized {len(blog_chunks)} blog chunks in {time.time() - start} seconds.")

Uploaded and vectorized 1182 blog chunks in 111.69951510429382 seconds.


In [24]:
from pydantic import BaseModel, field_validator

class UpdatedPropertyValue(BaseModel):
    property_value: str

    @field_validator("property_value")
    def check_starts_with(cls, v):
        invalid_prefixes = [
            "Property Name",
            "References",
            "Instruction",
            "Property Value",
        ]
        for prefix in invalid_prefixes:
            if v.startswith(prefix):
                raise ValueError(f"property_value cannot start with '{prefix}'")
        return v


class UpdateProperty(dspy.Signature):
    """Your task is to generate the value of a property by following the given instruction
    using the provided name-value property references as context. VERY IMPORTANT!! IT IS EXTREMELY IMPORTANT THAT YOU ONLY OUTPUT THIS VALUE AND NOTHING ELSE!!! PLEASE DO NOT BEGIN THE RESPONSE WITH SOMETHING LIKE "Sure, I can help you with that!" OR AFTER THE RESPONSE WRITE AN EXPLANATION OF WHY YOU RESPONDED WITH THIS VALUE. ONLY OUTPUT THE PROPERTY VALUE!!"""

    property_name: str = dspy.InputField(
        desc="The name of the property that you should update."
    )
    references: str = dspy.InputField(
        desc="The name-value property pairs that you should refer to while updating the property."
    )
    instruction: str = dspy.InputField(
        desc="The instruction that you should strictly follow while updating the property."
    )
    property_value: UpdatedPropertyValue = dspy.OutputField(
        desc="VERY IMPORTANT!! ONLY OUTPUT THIS VALUE AND *NOTHING* ELSE!!!"
    )


class Program(dspy.Module):
    def __init__(self) -> None:
        self.predict = dspy.Predict(UpdateProperty)

    def forward(self, property_name: str, references: str, instruction: str) -> str:
        prediction = self.predict(
            property_name=property_name, references=references, instruction=instruction
        ).property_value
        return dspy.Prediction(property_value=prediction)

In [25]:
program = Program()

instruction = """
Given a snippet from a blog post published by Weaviate, a Vector Database company, construct a question that delves deeply into the underlying concepts and explores new dimensions beyond the provided information. 
VERY IMPORTANT!! These queries should emphasize the advantages of semantic search with vector embeddings over traditional keyword-based methods like BM25. Ensure that the query explores related concepts, implications, or applications without directly repeating any keywords from the source document.
"""

property_name="inspired_query"

In [26]:
import random

def create_training_set(num_samples, uuids, weaviate_index, property_name, instruction):
    training_set = []
    for i in range(num_samples):
        uuid = random.choice(uuids)
        obj = weaviate_index.query.fetch_object_by_id(uuid)
        references = " ".join(f"{k}: {v}" for k, v in obj.properties.items())
        training_set.append(dspy.Example(
            property_name=property_name,
            references=references,
            instruction=instruction
        ).with_inputs('property_name', 'references', 'instruction'))
    return training_set

trainset = create_training_set(
    num_samples=20,
    uuids=blog_chunk_uuids,
    weaviate_index=blogs,
    property_name=property_name,
    instruction=instruction
)

In [27]:
class TypedGFLEvaluator(dspy.Signature):
    """A system has been given the assignment to generate the value of a property by following the instruction using the provided name-value property references.
Evaluate how well the response follows the instruction on a scale of 1 to 5. Please note it is EXTREMELY IMPORTANT that the property_value strictly contains the value and nothing else in the response. If it does have something other than the value itself, please rate it very low."""
    
    property_value: str = dspy.InputField(
        desc="The response from the system."
    )
    property_name = dspy.InputField(
        desc="The name of the property that you should update."
    )
    references = dspy.InputField(
        desc="The name-value property pairs that you should refer to while updating the property."
    )
    instruction = dspy.InputField(
        desc="The prompt to use when generating the updated property value."
    )
    rating: float = dspy.OutputField(desc="A float rating between 1 and 5. IMPORTANT!! ONLY OUTPUT THE RATING!!")

class OnlyOutputsPropertyValue(dspy.Signature):
    """It is extremely important that the response only contains the `property_value` and no meta information or reflections about the assignment. Please output whether or not the `property_value` does this on a integer valued scale of 0 to 1."""

    property_value: str = dspy.InputField(
        desc="The response from the system containing the property value."
    )
    rating: int = dspy.OutputField(desc="An integer rating of either 0 or 1. IMPORTANT!! ONLY OUTPUT THE RATING!!")

def MetricWrapper(gold, pred, trace=None):
    try:
        # Evaluate the quality of the instruction following
        quality_result = dspy.TypedPredictor(TypedGFLEvaluator)(
            property_value=pred.property_value,
            property_name=gold.property_name,
            references=gold.references,
            instruction=gold.instruction
        )
        quality_rating = float(quality_result.rating)
        quality_rating = max(0, min(5, quality_rating))

        # Evaluate if only the property value is output
        only_value_result = dspy.TypedPredictor(OnlyOutputsPropertyValue)(
            property_value=pred.property_value
        )
        only_value_rating = int(only_value_result.rating)

        # Combine the ratings with the specified weighting
        # The OnlyOutputsPropertyValue rating is 5 times more important
        combined_rating = (quality_rating + 5 * only_value_rating * 5) / 6

        return combined_rating
    except ValueError:
        # If conversion fails, return a default middle score
        print(f"Warning: Unable to parse rating.")
        return 2.5

# Hypothesis: Turn off Examples

In [43]:
from dspy.teleprompt import MIPROv2

teleprompter = MIPROv2(prompt_model=gpt4, task_model=llama3_ollama, metric=MetricWrapper, num_candidates=10, init_temperature=1.0)
kwargs = dict(num_threads=1, display_progress=True, display_table=0)
MIPRO_compiled_GFL = teleprompter.compile(program, trainset=trainset[:10], num_batches=10, max_bootstrapped_demos=0, max_labeled_demos=0, eval_kwargs=kwargs)


Please be advised that based on the parameters you have set, the maximum number of LM calls is projected as follows:


- Prompt Model: 10 data summarizer calls + 10 * 1 lm calls in program + (2) lm calls in program aware proposer = 22 prompt model calls
- Task Model: 25 examples in minibatch * 10 batches + 10 examples in train set * 1 full evals = 260 task model calls

Estimated Cost Calculation:

Total Cost = (Number of calls to task model * (Avg Input Token Length per Call * Task Model Price per Input Token + Avg Output Token Length per Call * Task Model Price per Output Token) 
            + (Number of calls to prompt model * (Avg Input Token Length per Call * Task Prompt Price per Input Token + Avg Output Token Length per Call * Prompt Model Price per Output Token).

For a preliminary estimate of potential costs, we recommend you perform your own calculations based on the task
and prompt models you intend to use. If the projected costs exceed your budget or expectations, you may c

 30%|█████████████▏                              | 3/10 [00:18<00:43,  6.26s/it]
[I 2024-06-27 08:28:18,404] A new study created in memory with name: no-name-38108567-9ad2-4a22-9074-4da4a4d57202


Using a randomly generated configuration for our grounded proposer.
Selected tip: simple
PROGRAM DESCRIPTION: 'Current Location'
References: 'Initial Location: New York, Current Location: Boston, Final Location: Miami'
Instruction: 'Move two steps South and update the current location'

Result:
Property Value: 'Washington D.C.'

SUMMARY OF PROGRAM ABOVE:
The program appears to be designed to solve tasks that require updating property values based on provided references and instructions. The program accomplishes this by using a language model to interpret and execute the instruction, using the references and property name as context. The property name identifies which property to update, the references provide name-value pairs for context, and the instruction provides a command to follow. The output is the new value of the specified property, following the instruction.

For example, given the property name 'Current Location', and a reference that states the 'Initial Location' was 'New Y

Average Metric: 9.5 / 10  (95.0): 100%|█████████| 10/10 [01:04<00:00,  6.48s/it]
FULL TRACE
Average Metric: 0.75 / 1  (75.0): 100%|███████████| 1/1 [00:06<00:00,  6.87s/it]



It is extremely important that the response only contains the `property_value` and no meta information or reflections about the assignment. Please output whether or not the `property_value` does this on a integer valued scale of 0 to 1.

---

Follow the following format.

Property Value: The response from the system containing the property value.
Rating: An integer rating of either 0 or 1. IMPORTANT!! ONLY OUTPUT THE RATING!! (Respond with a single int value)

---

Property Value: How can collaborative filtering and knowledge graph embeddings enhance the reliability of a vector database by leveraging collective intelligence and uncovering hidden relationships between entities?
Rating: 0






It is extremely important that the response only contains the `property_value` and no meta information or reflections abou

[I 2024-06-27 08:31:38,480] Trial 0 finished with value: 95.0 and parameters: {'0_predictor_instruction': 1}. Best is trial 0 with value: 95.0.


...





It is extremely important that the response only contains the `property_value` and no meta information or reflections about the assignment. Please output whether or not the `property_value` does this on a integer valued scale of 0 to 1.

---

Follow the following format.

Property Value: The response from the system containing the property value.
Rating: An integer rating of either 0 or 1. IMPORTANT!! ONLY OUTPUT THE RATING!! (Respond with a single int value)

---

Property Value: How can collaborative filtering and knowledge graph embeddings enhance the reliability of a vector database by leveraging collective intelligence and uncovering hidden relationships between entities?
Rating: 0






It is extremely important that the response only contains the `property_value` and no meta information or reflections about the assignment. Please output whether or not the `property_value` does this on a integer valued scale of 0 to 1.

---

Follow the following format.

Property Value: The r

[I 2024-06-27 08:41:37,240] Trial 8 finished with value: 216.67 and parameters: {'0_predictor_instruction': 3}. Best is trial 6 with value: 354.17.






It is extremely important that the response only contains the `property_value` and no meta information or reflections about the assignment. Please output whether or not the `property_value` does this on a integer valued scale of 0 to 1.

---

Follow the following format.

Property Value: The response from the system containing the property value.
Rating: An integer rating of either 0 or 1. IMPORTANT!! ONLY OUTPUT THE RATING!! (Respond with a single int value)

---

Property Value: How do the collaborative efforts and collective knowledge of a community contribute to the development of more accurate and reliable semantic search models, ultimately enhancing the overall performance and value of vector-based databases like Weaviate?
Rating: 0






It is extremely important that the response only contains the `property_value` and no meta information or reflections about the assignment. Please output whether or not the `property_value` does this on a integer valued scale of 0 to 1.

---

[I 2024-06-27 08:42:51,612] Trial 9 finished with value: 178.33 and parameters: {'0_predictor_instruction': 8}. Best is trial 6 with value: 354.17.






It is extremely important that the response only contains the `property_value` and no meta information or reflections about the assignment. Please output whether or not the `property_value` does this on a integer valued scale of 0 to 1.

---

Follow the following format.

Property Value: The response from the system containing the property value.
Rating: An integer rating of either 0 or 1. IMPORTANT!! ONLY OUTPUT THE RATING!! (Respond with a single int value)

---

Property Value: How can the integration of graph-based reasoning and vector embeddings in a Vector Database like Weaviate enable more accurate and context-aware recommendations for users, ultimately enhancing the overall search experience?
Rating: 0






It is extremely important that the response only contains the `property_value` and no meta information or reflections about the assignment. Please output whether or not the `property_value` does this on a integer valued scale of 0 to 1.

---

Follow the following format

In [44]:
uuid = random.choice(blog_chunk_uuids)
obj = blogs.query.fetch_object_by_id(uuid)
references = " ".join(f"{k}: {v}" for k, v in obj.properties.items())

'''
Although the context manager is unnecessary for this notebook,
-- I left it there to remind people that you might have
-- set llama3 or another model as the default. So be careful if you
-- optimized the prompt for mistral, qwen, or what have you and then
-- are testing with a different model.
'''

with dspy.context(lm=llama3_ollama):
    print(MIPRO_compiled_GFL(
        property_name=property_name,
        instruction=instruction,
        references=references
    ))

Prediction(
    property_value='How do multimodal embedding models enable more accurate and comprehensive knowledge retrieval in complex domains by leveraging the synergies between different sensory inputs?'
)


In [46]:
llama3_ollama.inspect_history(n=1)




Based on the property name, provided references, and instructions, generate a suitable property value. Remember, the output should only be this property value and not include unnecessary explanations or comments. The product of this task should appear as an embodiment of the instruction, processed through the contextual lens of the references.

---

Follow the following format.

Property Name: The name of the property that you should update.

References: The name-value property pairs that you should refer to while updating the property.

Instruction: The instruction that you should strictly follow while updating the property.

Property Value: VERY IMPORTANT!! ONLY OUTPUT THIS VALUE AND *NOTHING* ELSE!!!

---

Property Name: inspired_query

References: inspired_question: None content: --- title: Multimodal Embedding Models slug: multimodal-models authors: zain date: 2023-06-27 image: ./img/hero.png tags: ['concepts'] description: "ML Models that can see, read, hear and more!" --- ![M

'\n\n\nBased on the property name, provided references, and instructions, generate a suitable property value. Remember, the output should only be this property value and not include unnecessary explanations or comments. The product of this task should appear as an embodiment of the instruction, processed through the contextual lens of the references.\n\n---\n\nFollow the following format.\n\nProperty Name: The name of the property that you should update.\n\nReferences: The name-value property pairs that you should refer to while updating the property.\n\nInstruction: The instruction that you should strictly follow while updating the property.\n\nProperty Value: VERY IMPORTANT!! ONLY OUTPUT THIS VALUE AND *NOTHING* ELSE!!!\n\n---\n\nProperty Name: inspired_query\n\nReferences: inspired_question: None content: --- title: Multimodal Embedding Models slug: multimodal-models authors: zain date: 2023-06-27 image: ./img/hero.png tags: [\'concepts\'] description: "ML Models that can see, read,